## Notebook for spike sorting from .rhd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info('Logger set')

2018-09-30 01:34:16,035 - root - INFO - Logger set


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
raw_folder = os.path.abspath('/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01')

In [4]:
exp_base = os.path.abspath('/data/experiment/')

file_sys = {'exp_base': os.path.abspath('/data/experiment/')}

sess_par = {'bird': 'o3g3',
           'sess': 'awake_20180908_2925_01',
           'rec': 0}

file_sys_defaults = {'exp_base': os.path.abspath('/data/experiment')}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [5]:
exp_base

'/data/experiment'

In [6]:
raw_folder = os.path.join(exp_base, 'raw_data', sess_par['bird'], 'neural', sess_par['sess'])
logger.info('Raw folder {}'.format(raw_folder))
glob.glob(os.path.join(raw_folder, '*.rhd'))

2018-09-30 01:34:20,430 - root - INFO - Raw folder /data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01


['/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180909_053538.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180909_015538.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180909_025538.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_201538.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180909_014538.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_173537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180909_043538.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_172537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_180537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180909_081538.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_int

In [7]:
rhd_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
rhd_files.sort()
pd_f = pd.DataFrame(rhd_files, columns=['f_path'])

pd_f['stamp'] = pd_f['f_path'].apply(lambda x: int(os.path.split(x)[-1].split('.')[0].split('_')[-1]))

# Select some of the files
sess_start = 0
sess_end = 235959
pd_session = pd_f.loc[(pd_f['stamp'] > sess_start) & (pd_f['stamp'] < sess_end), :]
pd_session.index.size

rhd_list = pd_session['f_path'].values.tolist()[:]

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [8]:
rhd_list

['/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_172537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_173537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_174537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_175537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_180537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_181537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_182537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_183537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_184537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_185537.rhd',
 '/data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_int

In [9]:
ss_folder = os.path.join(exp_base, 'ss_data', sess_par['bird'], sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al
s_f = 20000
ds_params = {'samplerate': s_f}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
intan_meta, samples_in = intan.intan_to_bin(rhd_list[:], file_paths['mda'], variant='mda')


2018-09-30 01:35:34,622 - root - INFO - Created session par files /data/experiment/ss_data/o3g3/awake_20180908_2925_01/params.json
2018-09-30 01:35:34,626 - pipefinch.neural.convert - INFO - Creating the output file /data/experiment/ss_data/o3g3/awake_20180908_2925_01/raw.mda
2018-09-30 01:35:34,627 - pipefinch.neural.convert - INFO - File 0/102
2018-09-30 01:35:34,627 - intan2kwik.intan.load_intan_jit - INFO - Reading rhd intan file /data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_172537.rhd
notch filter: 100%|██████████| 17/17 [00:02<00:00,  5.81it/s]
2018-09-30 01:36:20,620 - pipefinch.neural.convert - INFO - File 1/102
2018-09-30 01:36:20,625 - intan2kwik.intan.load_intan_jit - INFO - Reading rhd intan file /data/experiment/raw_data/o3g3/neural/awake_20180908_2925_01/raw_intan_180908_173537.rhd
notch filter: 100%|██████████| 17/17 [00:02<00:00,  5.83it/s]
2018-09-30 01:37:06,572 - pipefinch.neural.convert - INFO - File 2/102
2018-09-30 01:37:06,573 - in

In [10]:
ss_folder

'/data/experiment/ss_data/o3g3/awake_20180908_2925_01'

In [ ]:
file_paths

In [ ]:
msc.read_dataset_params(ss_folder)

In [18]:
reload(msc)

<module 'pipefinch.neural.sort.mountain.core' from '/home/ezequiel/repos/pipefinch/pipefinch/neural/sort/mountain/core.py'>

In [11]:
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline()


JSProxyWidget(status='Not yet rendered')

In [12]:
output_dir = os.path.join(ss_folder, 'sort_out')
with Pipeline:
    msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

#msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1,detect_threshold=3)

2018-09-30 02:59:23,589 - pipefinch.sort.mountain.comre - INFO - Bandpass filter
2018-09-30 02:59:23,595 - pipefinch.sort.mountain.comre - INFO - Whitening
2018-09-30 02:59:23,597 - pipefinch.sort.mountain.comre - INFO - Sorting
/home/ezequiel/repos/pipefinch/pipefinch/neural/sort/mountain/core.py:168: UserWarning: Will sort with no geometry input
  warnings.warn('Will sort with no geometry input')
2018-09-30 02:59:23,599 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics
2018-09-30 02:59:23,603 - pipefinch.sort.mountain.comre - INFO - Automatically curating


Output()

<details />

Exception: [Click to expand] Error running process ephys.bandpass_filter: Process returned with non-zero error code (255)

In [33]:
file_paths

{'mda': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda',
 'params': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/params.json',
 'geom': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/geom.csv',
 'filt': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json'}

In [10]:
cmd = 'env'
p = subprocess.Popen([cmd], stdout=subprocess.PIPE)
ans = p.communicate()
print(ans)

(b'MANPATH=/opt/local/share/man/:\nTERM_PROGRAM=Apple_Terminal\nSHELL=/bin/bash\nTERM=xterm-color\nTMPDIR=/var/folders/gk/7z94s6yj40n4ggn5dl62xhbc0000gn/T/\nCONDA_SHLVL=1\nApple_PubSub_Socket_Render=/private/tmp/com.apple.launchd.rxM5aew32h/Render\nCONDA_PROMPT_MODIFIER=(mountain) \nTERM_PROGRAM_VERSION=404\nTERM_SESSION_ID=E6D6F9C0-AA53-473E-B5D3-B4EE02E408C3\nUSER=zeke\nCONDA_EXE=/Users/zeke/anaconda3/bin/conda\nML_CONDA_PACKAGES_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages\nSSH_AUTH_SOCK=/private/tmp/com.apple.launchd.ZGXuue9qT4/Listeners\nML_CONDA_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nPATH=/Users/zeke/anaconda3/envs/mountain/bin:/Users/zeke/google-cloud-sdk/bin:/opt/local/bin:/opt/local/sbin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin\nCONDA_PREFIX=/Users/zeke/anaconda3/envs/mountain\nPWD=/Users/zeke/repos\nML_CONFIG_DIRECTORY=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nLANG=en_US

In [16]:
show_args_dict = {'raw': file_paths['mda'], 
             'filt': file_paths['filt'],
             'pre': file_paths['pre'],
            'samplereate': ds_params['samplerate'],
            'firings': file_paths['firings_curated'],
            'cluster_metrics': file_paths['cluster_metrics']}

cmd_args = ['--{} {}'.format(k, v) for k, v in show_args_dict.items()]
cmd = 'qt-mountainview'

logger.info('View command is {}'.format([cmd, ' '.join(cmd_args)]))
p = subprocess.Popen([cmd, ' '.join(cmd_args)], stdout=subprocess.PIPE)


2018-10-01 17:34:53,025 - root - INFO - View command is ['qt-mountainview', '--raw /data/experiment/ss_data/o3g3/awake_20180908_2925_01/raw.mda --filt /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/filt.mda.prv --pre /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/pre.mda.prv --samplereate 20000 --firings /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/firings_curated.mda --cluster_metrics /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/cluster_metrics.json']


In [17]:
print(' '.join(['qt-mountainview'] + cmd_args))

qt-mountainview --raw /data/experiment/ss_data/o3g3/awake_20180908_2925_01/raw.mda --filt /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/filt.mda.prv --pre /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/pre.mda.prv --samplereate 20000 --firings /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/firings_curated.mda --cluster_metrics /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/cluster_metrics.json


# other viewing examples

ev-templates output/templates.mda.prv
ev-timeseries dataset/raw.mda.prv --firings output/firings.mda.prv --samplerate=30000
qt-mountainview --raw dataset/raw.mda.prv --filt output/filt.mda.prv --pre output/pre.mda.prv --samplerate 30000 --firings output/firings.mda.prv
